In [1]:
import sys, os; sys.path.append('..')
import pyzx as zx
import random
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [6]:
c = zx.qasm("""
qreg q[4];
h q[0];
rz(0.125*pi) q[2];
s q[0];
cx q[2],q[3];
cx q[1],q[2];
x q[3];
cx q[0],q[1];
cx q[2],q[3];
tdg q[1];
cx q[0],q[1];
cx q[1],q[2];
rz(-0.125*pi) q[2];
z q[0];
""")

zx.draw(c)

In [7]:
g = c.to_graph()
zx.full_reduce(g)
zx.draw(g)

In [59]:
random.seed(1337)
c = zx.generate.CNOT_HAD_PHASE_circuit(4, 100, p_had=0)
# c.gates.append(zx.gates.HAD(0))
# c.gates.append(zx.gates.HAD(1))
# c.gates.append(zx.gates.HAD(2))
# c.gates.append(zx.gates.HAD(3))
# c.add_circuit(zx.generate.CNOT_HAD_PHASE_circuit(4, 100, p_had=0))
zx.draw(c)

In [65]:
g = c.to_graph()
zx.full_reduce(g)
g.normalise()
g = g.copy()
zx.draw(g,labels=True)

In [74]:
[g.neighbours(v) for v in [21,10,12,11]]

[dict_keys([13, 18]),
 dict_keys([6, 8, 9, 14, 11]),
 dict_keys([4, 15, 17]),
 dict_keys([4, 10, 16, 18])]

In [ ]:
zx.extract.bi_adj([21,10,12,11])

In [61]:
ge = g.copy()
ce = zx.extract_circuit(ge)
zx.draw(ce)

In [58]:
g1 = g.copy()
g1.normalise()
# squash phases on boundary
# g1.set_phase(14, 0)
# g1.set_phase(13, 0)

# pivot away gadgets
#zx.rules.apply_rule(g1,zx.rules.pivot,[(9,14,[],[17])])

zx.draw(g1,labels=True)

In [41]:
g = zx.Graph()

greens = 4
reds = 3
for i in range(greens):
    g.add_vertex(ty=1, qubit=i, row=1)
for i in range(reds):
    v = g.add_vertex(ty=2, qubit=i, row=3)
    g.set_phase(v, 1)

random.seed(1337)
p_edge = 0.5
for i in range(greens):
    for j in range(greens, greens+reds):
        if random.random() < p_edge:
            g.add_edge((i,j))
zx.draw(g)

In [42]:
zx.full_reduce(g)

In [43]:
g.scalar

Scalar(2.83+0.00i = sqrt(2)^1(1+exp(0ipi)))

In [2]:
g = zx.qasm("""
qreg q[3];

ccz q[0],q[1],q[2];
h q[2];
t q[2];
ccz q[0],q[1],q[2];
h q[2];
t q[1];
ccz q[0],q[1],q[2];
s q[2];
ccx q[0],q[1],q[2];
""").to_graph(zh=True)
zx.draw(g, labels=True)

In [ ]:
h = g.copy()
zx.simplify.spider_simp(h)
zx.hsimplify.to_hbox(h)
m = zx.hrules.match_hpivot(h)
print(m)
display(zx.draw(h,labels=True))
zx.hrules.hpivot(h,m)
display(zx.draw(h,labels=True))

In [ ]:
zx.hsimplify.hpivot_simp(g)
zx.d3.draw(g, labels=True)

In [19]:
qs = 10
d = 60
c = zx.Circuit(qs)
p_t = 0
p_h = 0.3
random.seed(1338)
for i in range(d):
    p = random.random()
    if p < p_h:
        c.add_gate(zx.circuit.HAD(random.randint(0,qs-1)))
    elif p < p_t + p_h:
        c.add_gate(zx.circuit.T(random.randint(0,qs-1)))
    else:
        q = sorted(random.sample(range(qs), 3))
        c.add_gate(zx.circuit.CCZ(q[0],q[1],q[2]))

In [ ]:
zx.d3.draw(c.to_graph(zh=True))

In [20]:
g = c.to_graph(zh=True)
zx.hsimplify.hpivot_simp(g)
g.normalise()
zx.d3.draw(g)

spider_simp: 42. 26. 20. 12. 10. 9. 6. 1.  8 iterations
id_simp: 4.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 7.  1 iterations
spider_simp: 1. 1.  2 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations


In [16]:
[len([v for v in g.vertices() if g.type(v) == t]) for t in [1,3]]

[16, 63]

In [ ]:
[max([g.vertex_degree(v) for v in g.vertices() if g.type(v) == t]) for t in [1,3]]

In [ ]:
zx.compare_tensors(c.to_graph(zh=True), g)

In [ ]:
d = { 'a': 3, 'b': 4, 'c': 3}
list(filter(lambda v: v > 3, d.values()))

In [ ]:
a = [1,2,3]
a.append(4)
a

In [ ]:
2**9

In [33]:
c1 = zx.Circuit.load('../circuits/Fast/QFT8_before')
c1

Circuit(8 qubits, 148 gates)

In [44]:
g = c1.to_graph(zh=True)
zx.hsimplify.hpivot_simp(g)
g.normalise()
print("Normal form: ", end='')
print(g.qubit_count() * 2 == len([v for v in g.vertices() if g.type(v) == 1]))
zx.d3.draw(g)

spider_simp: 24. 15. 11. 10. 6. 4. 4. 3.  8 iterations
id_simp: 8.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
spider_simp: 3. 3.  2 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 3.  1 iterations
spider_simp: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 6.  1 iterations
spider_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 3.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 9.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 4.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 6.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 6.  1 iterations
spider_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 22.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 11.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 3.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 4.  1

In [35]:
[len([v for v in g.vertices() if g.type(v) == t]) for t in [1,3]]

[16, 28]

In [40]:
g.qubit_count() * 2 == len([v for v in g.vertices() if g.type(v) == 1])

True

In [ ]:
os.path.join('..', 'circuits', 'Fast')
for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('before') != -1: